In [1]:
import pandas as pd
import numpy as np
import json
import os
import datetime
import requests
import tarfile
import sys
import csv
import importlib

# Add the parent directory to the system path so the notebook can find the eBird2ABAP package
sys.path.append(os.path.abspath("../"))

from eBird2ABAP import *

# Reload the module to apply changes
import eBird2ABAP

importlib.reload(eBird2ABAP)

<module 'eBird2ABAP' from '/Users/rafnuss/Library/CloudStorage/OneDrive-Vogelwarte/eBird2ABAP/eBird2ABAP/__init__.py'>

In [2]:
# Uninstall the local version
# !pip uninstall eBird2ABAP -y

In [2]:
# Download the file
# EBD_file = download_EBD()
# default is to load the latest file otherwise, you can specify the month and year
# file = download_EBD(year="2024", month="Jul")
EBD_file = "../data/eBird/ebd_AFR_relOct-2024/ebd_AFR_relOct-2024.txt.gz"

In [3]:
# Run everything
ebird2abap(EBD_file, exportCSV=True)

Reading EBD file...
Adding ADU number...
/Users/rafnuss/Library/CloudStorage/OneDrive-Vogelwarte/eBird2ABAP/eBird2ABAP/matched_species.csv
Computing checklists...
Checking validity of cards...
Converting valid cards to CHK cards...
Converting CHK cards to card check...
Converting CHK cards to EBD formatted units...
Converting EBD formatted units to card expressions...
Converting card expressions to JSON format...
Writing JSON data to ebd_AFR_relOct-2024_20241115_113329.json...
Writing CSV data...
Process completed successfully.


In [5]:
ebd = read_EBD(EBD_file)

In [ ]:
# ebd0[ebd0["SAMPLING EVENT IDENTIFIER"] == "S14210192"]
ebd[ebd["TAXON CONCEPT ID"] == "avibase-732D08C3"]

,CATEGORY,TAXON CONCEPT ID,SCIENTIFIC NAME,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,SAMPLING EVENT IDENTIFIER,PROTOCOL TYPE,DURATION MINUTES,EFFORT DISTANCE KM,ALL SPECIES REPORTED
92419,species,avibase-732D08C3,Gallirallus philippensis,-2.000028,35.169404,1999-03-01,08:00:00,obsr292352,S151510263,Stationary,120.0,NaN,1


In [5]:
add_ADU(ebd0, return_unmatched=True)

/Users/rafnuss/anaconda3/lib/python3.11/site-packages/eBird2ABAP/matched_species.csv
We still have 0 unmatched taxons, corresponding to 0% of our data


Series([], Name: count, dtype: int64)

In [4]:
ebd0 = ebd1

In [5]:
ebd = add_ADU(ebd0)
print(f"# data: {len(ebd)}")
print(f"# Checklists: {len(ebd['SAMPLING EVENT IDENTIFIER'].unique())}")
print(f"# Species: {len(ebd['SCIENTIFIC NAME'].unique())}")

/Users/rafnuss/anaconda3/lib/python3.11/site-packages/eBird2ABAP/matched_species.csv
# data: 21190167
# Checklists: 1262874
# Species: 3127


In [9]:
ebd[ebd["SAMPLING EVENT IDENTIFIER"] == "S14210192"]

,CATEGORY,TAXON CONCEPT ID,SCIENTIFIC NAME,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,SAMPLING EVENT IDENTIFIER,PROTOCOL TYPE,DURATION MINUTES,EFFORT DISTANCE KM,ALL SPECIES REPORTED,OBSERVATION DATETIME,ADU
3320183,species,avibase-02B06535,Anthobaphes violacea,-34.353312,18.489614,2013-01-04,14:00:00,obsr88127,S14210192,Stationary,120.0,NaN,1,2013-01-04 14:00:00,753.0


In [7]:
chk = ebd2chk(ebd)

In [8]:
card_valid = chk2valid_card(chk)

In [9]:
chk_card = valid_card2chk_card(chk, card_valid)

In [10]:
card_chk = chk_card2card_chk(chk_card, card_valid)

In [11]:
ebd_f_u = chk_card2ebd_f_u(ebd, chk_card)

In [12]:
card_exp = ebd_f_u2card_exp(card_chk, ebd_f_u)

In [13]:
len(card_exp)

62396

In [14]:
json_data = card_exp.to_json(orient="records", indent=2)
with open(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
    "w",
) as f:
    f.write(json_data)

In [15]:
card_chk.to_csv(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_cards.csv",
    index=False,
)

In [16]:
ebd_f_u[["CARD", "ADU", "SEQ"]].to_csv(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_records.csv",
    index=False,
)